# Volterra for different solvers

In case you made yourself familiar with the method setup in the main main notebook `volterra.ipynb` then you can check out the functionality of different solvers here. 

The operator is now imported from the `volterra.py` file. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from regpy.operators import Operator
from regpy.vecsps import UniformGridFcts
from volterra import Volterra

###  Defining the exact solution

In [ ]:
grid = UniformGridFcts(np.linspace(0, 2 * np.pi, 200))

exact_solution = (1-np.cos(grid.coords[0]))**2/4 
plt.figure()
plt.plot(grid.coords[0],exact_solution,color="blue",label='exact solution')
plt.legend()
plt.show()

## Regularization Methods

In [ ]:
from regpy.hilbert import L2, Sobolev
from regpy.functionals import HilbertNorm, TV
from regpy.solvers import RegularizationSetting, TikhonovRegularizationSetting
import regpy.stoprules as rules

## Tikhonov

The Tikhonov method for linear inverse problems. Minimizes
$$
    \Vert T x - g^\delta\Vert^2 + \alpha * \Vert x - x_{ref}\Vert^2
$$
using a conjugate gradient method.

Requires a linear operator hence we use Volterra with exponent $n=1$. As a regularization setting use $L^2$ both as penalty and data fidelity. 

In [ ]:
from regpy.solvers.linear.tikhonov import TikhonovCG

op = Volterra(grid)

exact_data = op(exact_solution)
noise = 0.03 * op.domain.randn()
data = exact_data + noise

setting = RegularizationSetting(op, L2, L2)

solver = TikhonovCG(setting, data, regpar=0.01)
stoprule = (
    rules.CountIterations(1000) +
    rules.Discrepancy(
        setting.h_codomain.norm, data,
        noiselevel=setting.h_codomain.norm(noise),
        tau=1.1
    )
)

reco, reco_data = solver.run(stoprule)

fig, axs = plt.subplots(nrows=1, ncols=1)
axs.plot(grid.coords[0], reco,color="lightblue",label="reconstruction")
axs.plot(grid.coords[0], reco_data,color="orange",label="reconstruction data")
axs.plot(grid.coords[0], exact_solution,color="blue",label="exact solution")
axs.plot(grid.coords[0], exact_data, color="green",label="exact data")
axs.plot(grid.coords[0], data, color="lightgreen",label="noisy data")
plt.legend()
plt.show()

## Landweber Iteration 

The Landweber method. Solves the potentially non-linear, ill-posed equation
$$
    T(x) = g^\delta,
$$
where $T$ is a Frechet-differentiable operator, by gradient descent for the residual
$$
    \Vert T(x) - g^\delta\Vert^2,
$$
where $\Vert\cdot\Vert$ is the Hilbert space norm in the codomain.

Here we use a standard Sobolev norm on the domain as penalty term since the exact solution is smooth. 

In [ ]:
from regpy.solvers.nonlinear.landweber import Landweber

op = Volterra(grid,exponent=2)

exact_data = op(exact_solution)
noise = 0.03 * op.domain.randn()
data = exact_data + noise
init = op.domain.ones()*0.05

setting = RegularizationSetting(op, Sobolev, L2)

solver = Landweber(setting, data, init, stepsize=0.01)
stoprule = (
    # Landweber is slow, so need to use large number of iterations
    rules.CountIterations(max_iterations=100000) +
    rules.Discrepancy(
        setting.h_codomain.norm, data,
        noiselevel=setting.h_codomain.norm(noise),
        tau=1.1
    )
)

reco, reco_data = solver.run(stoprule)

fig, axs = plt.subplots(nrows=1, ncols=1)
axs.plot(grid.coords[0], reco,color="lightblue",label="reconstruction")
axs.plot(grid.coords[0], reco_data,color="orange",label="reconstruction data")
axs.plot(grid.coords[0], exact_solution,color="blue",label="exact solution")
axs.plot(grid.coords[0], exact_data, color="green",label="exact data")
axs.plot(grid.coords[0], data, color="lightgreen",label="noisy data")
plt.legend()
plt.show()

## FISTA

The generalized FISTA algorithm for minimization of Tikhonov functionals
$$
\mathcal{S}_{g^{\delta}}(F(f)) + \alpha \mathcal{R}(f).
$$ 
Gradient steps are performed on the first term, and proximal steps on the second term. 

FISTA is implemented with Impulsive noise in the middle of the interval. For a good reconstruction even with the impulsive noise use the `TV` penalty with `Sobolev` norm as penalty. For the data fidelity use the standard `L2` data fidelity. Note that this example demonstrates that you can define `AbstractFunctional` on Hilbertspace of type `AbstractSpace` by simply writing `HilbertNorm(h_space= AbstractSpace)` and you may compose this instances with operators by multiplying the operator from the right side. 

In [ ]:
from regpy.solvers.nonlinear.fista import FISTA

op = Volterra(grid, exponent=2)

# Impulsive Noise
sigma = 0.01*np.ones(grid.coords.shape[1])
sigma[100:110] = 0.5

exact_data = op(exact_solution)
noise = sigma * op.domain.randn()
data = exact_data + noise
init = op.domain.ones()

#The penalty term |f|_{TV}
setting = TikhonovRegularizationSetting(
    op=op, 
    penalty=TV(h_domain=Sobolev), 
    data_fid=HilbertNorm(h_space=L2), 
    data_fid_shift = data,
    regpar = 0.01
)

proximal_pars = {
        'stepsize' : 0.001,
        'maxiter' : 100
        }
# """Parameters for the inner computation of the proximal operator with the Chambolle algorithm"""


solver = FISTA(setting, init=init, proximal_pars=proximal_pars)
stoprule = (
    # Method is slow, so need to use large number of iterations
    rules.CountIterations(max_iterations=100) +
    rules.Discrepancy(
        setting.h_codomain.norm, data,
        noiselevel=setting.h_codomain.norm(noise),
        tau=1.1
    )
)

reco, reco_data = solver.run(stoprule)

fig, axs = plt.subplots(nrows=1, ncols=1)
axs.plot(grid.coords[0], reco,color="lightblue",label="reconstruction")
axs.plot(grid.coords[0], reco_data,color="orange",label="reconstruction data")
axs.plot(grid.coords[0], exact_solution,color="blue",label="exact solution")
axs.plot(grid.coords[0], exact_data, color="green",label="exact data")
axs.plot(grid.coords[0], data, color="lightgreen",label="noisy data")
plt.legend()
plt.show()


## Semi-Smooth Newton with bilateral constraints

Semismooth Newton method for minimizing quadratic Tikhonov functionals
$$
    \Vert T x - g^\delta\Vert^2 + \alpha * \Vert x - x_{ref}\Vert^2
$$
subject to bilateral constraints $\psi_- \leq x \leq \psi_+$.


In [ ]:
from regpy.solvers.linear.semismoothNewton import SemismoothNewton_bilateral

op = Volterra(grid, exponent=1)
#solver requires linear Forward problem therefore choose exponent = 1

exact_data = op(exact_solution)
noise = 0.1 * op.domain.randn()
data = exact_data + noise
init = op.domain.ones()*0.0005

setting = RegularizationSetting(
    op=op, 
    penalty=L2, 
    data_fid=L2
)


solver = SemismoothNewton_bilateral(
    setting, 
    data, 
    0.3,
    xref = init,
    )

stoprule = (
    rules.CountIterations(max_iterations=100) +
    rules.Discrepancy(
        setting.h_codomain.norm, data,
        noiselevel=setting.h_codomain.norm(noise),
        tau=1.1
    )
)

reco, reco_data = solver.run(stoprule)

fig, axs = plt.subplots(nrows=1, ncols=1)
axs.plot(grid.coords[0], reco,color="lightblue",label="reconstruction")
axs.plot(grid.coords[0], reco_data,color="orange",label="reconstruction data")
axs.plot(grid.coords[0], exact_solution,color="blue",label="exact solution")
axs.plot(grid.coords[0], exact_data, color="green",label="exact data")
axs.plot(grid.coords[0], data, color="lightgreen",label="noisy data")
plt.legend()
plt.show()

## Semi-Smooth Newton Frozen with bilateral constraints for non-linear equations 

Similarly to the above method the 'NewtonSemiSmoothFrozen' uses the Semismooth Newton method for minimizing quadratic Tikhonov functionals
$$
    \Vert T x - g^\delta\Vert^2 + \alpha * \Vert x - x_{ref}\Vert^2
$$
subject to bilateral constraints $\psi_- \leq x \leq \psi_+$ in each step by linearizing the equation at the current iterate.  

In [ ]:
from regpy.solvers.nonlinear.newton import NewtonSemiSmoothFrozen

op = Volterra(grid, exponent=2)


exact_data = op(exact_solution)
noise = 0.1 * op.domain.randn()
data = exact_data + noise
init = op.domain.ones()*0.0005

setting = RegularizationSetting(
    op=op, 
    penalty=L2, 
    data_fid=L2
)


solver = NewtonSemiSmoothFrozen(
    setting, 
    data, 
    (1,0.9),
    op.domain.ones()*-2.5, 
    op.domain.ones()*2,
    init = init, 
    inner_NSS_iter_max=10)


stoprule = (
    rules.CountIterations(max_iterations=20) +
    rules.Discrepancy(
        setting.h_codomain.norm, 
        data,
        noiselevel=setting.h_codomain.norm(noise),
        tau=2.1
    )
)

reco, reco_data = solver.run(stoprule)

fig, axs = plt.subplots(nrows=1, ncols=1)
axs.plot(grid.coords[0], reco,color="lightblue",label="reconstruction")
axs.plot(grid.coords[0], reco_data,color="orange",label="reconstruction data")
axs.plot(grid.coords[0], exact_solution,color="blue",label="exact solution")
axs.plot(grid.coords[0], exact_data, color="green",label="exact data")
axs.plot(grid.coords[0], data, color="lightgreen",label="noisy data")
plt.legend()
plt.show()

## ADMM: Alternating direction method of multipliers

This method minimizes is used to find solutions to 
$$
    \min_{u,v} [\mathcal{F}(u)+\mathcal{G}(v)] \; \text{s.t.} \; Au+Bv=b
$$
Choosing 
$$
        A:=\begin{pmatrix} T \\ I \end{pmatrix} ,\;
        B:=\begin{pmatrix} -I & 0 \\ 0 & -I \end{pmatrix}, \;
        b:=\begin{pmatrix} 0 \\ 0 \end{pmatrix} ,\;
        F(f)\equiv 0,\;
        G\begin{pmatrix} v_1 \\ v_2 \end{pmatrix}:=R(v_1)+S(v_2) \;
$$
it can be used to minimize 
$$
    \argmin_f [S(Tf) + \alpha * R(f)]
$$
for linear operators $T$

In [ ]:
"""from regpy.solvers.linear.admm import ADMM

# Operator need to be linear 
op = Volterra(grid, exponent=1)

# Impulsive Noise
sigma = 0.3*np.ones(grid.coords.shape[1])
sigma[100:110] = 2

exact_data = op(exact_solution)
noise = sigma * op.domain.randn()
data = exact_data + noise
init = {
    "v1" : op.codomain.ones(),
    "v2" : op.domain.ones(),
    "p1" : op.codomain.ones(),
    "p2" : op.domain.ones(),
}

#construct the data misfit functional as combination of norm with Shifted operator.
from regpy.operators import Identity
setting = RegularizationSetting(
    op=op, 
    penalty=L2, 
    data_fid=HilbertNorm(h_space=L2) * (Identity(op.codomain) - data)
)

regpar = 0.01
gamma = 1

solver = ADMM(setting, init, gamma=gamma)
stoprule = (
    rules.CountIterations(max_iterations=10) +
    rules.Discrepancy(
        setting.h_codomain.norm, data,
        noiselevel=setting.h_codomain.norm(noise),
        tau=1.1
    )
)


reco, reco_data = solver.run(stoprule)

fig, axs = plt.subplots(nrows=1, ncols=1)
axs.plot(grid.coords[0], reco,color="lightblue",label="reconstruction")
axs.plot(grid.coords[0], reco_data,color="orange",label="reconstruction data")
axs.plot(grid.coords[0], exact_solution,color="blue",label="exact solution")
axs.plot(grid.coords[0], exact_data, color="green",label="exact data")
axs.plot(grid.coords[0], data, color="lightgreen",label="noisy data")
plt.legend()
plt.show()
"""